## Objetivo y alcance




Para esta materia el objetivo es poder hacer un primer acercamiento a un proceso de aprendizaje supervisado.

En este laboratorio no se espera que se encuentre el mejor modelo con sus mejores parámetros, sino que se logre la buena práctica de realizar los pasos necesarios en un proceso de aprendizaje supervisado, desde el entendimiento de la pregunta de investigación hasta la preparación de datos, desde la división del dataset hasta la evaluación del modelo. Para realizar el práctico vamos a utilizar el datasets que hayan generado en el práctico anterior.


## Introducción
En un mismo día une operador humano de una clínica responde cientos de consultas sobre problemáticas increiblemente sensibles como lo es la salud de una persona y las preocupaciones relacionadas a eso. Además la demanda de trabajo de un operador humano es multivariada y dinámica: algunos días hay un flujo mayor de pacientes usando los sistemas hospitalarios, y hay momentos del mes y del día en donde prácticamente no hay ninguna consulta a atender. 

El dato de **tiempo de respuesta de HH** tiene muchas implicaciones a nivel producto. Tener ese dato nos permite:
- informarle a la persona usuaria el tiempo promedio de respuesta antes de pasarla a hablar con une operador, para que adecúe sus expectativas a la realidad. 
- informarle detalladamente este dato a los hospitales como reporte de uso al final del mes, para que organice al grupo de trabajo eficientemente y con información veraz
- permitir configurar lógicas automáticas de repartición de tareas para les operadores, para facilitarles su trabajo

Por ello lo que se pretende en este práctico es poder armar un modelo que nos ayude a predecir el tiempo de respuesta de HH. Lo que nos da este dato es la presencia de la feature `Got_HH`, que cuando vale 1 nos marca en el dataset que el bot realizó la transferencia. El siguiente mensaje que tenga `Direction == outbound` sería el primer mensaje de un operador humano. La diferencia de horas entre estos dos mensajes nos da el tiempo de respuesta de un operador humano.

Predecir esta variable teniendo en cuenta las distintas features del dataset será una tarea de regresión. Entrenar un modelo por hospital dará una mejor predicción. Por conocimiento del dataset sabemos que entrenar por porción del día (mañana/tarde) también dará mejores resultados, pero esto es opcional.

## Tareas
En orden,

Deberán armar un dataset más pequeño que el actual, **sacando**:
- las filas de los hospitales sin HH (Demo, Desarrollo, Clínica Carrá, Default) y las conversaciones sin HH (que agrupadas por conv_id no tengan un Got_HH == 1)
- las conversaciones que nunca hayan pasado a HH (es decir Got_HH es 0 en toda la agrupación por conv_id) (guiarse con https://stackoverflow.com/questions/52393659/pandas-dataframe-check-if-column-value-exists-in-a-group-of-columns )
- Para simplificar, las conversaciones que hayan pasado a HH pero el operador nunca respondió. Es decir Got_HH es 1 en alguna fila, pero no hay una fila siguiente con direction igual a outbound.

**Generar** las features:

- `tiempo_de_respuesta_HH` (agrupando por conv_id, calcular el valor absoluto de la diferencia entre el SentDate del mensaje con Got_HH igual a 1 y el del siguiente mensaje con Direction igual a outbound https://stackoverflow.com/questions/34023918/make-new-column-in-panda-dataframe-by-adding-values-from-other-columns )
- cant de mensajes de la conversación (agrupación por conv_id+dia y luego sum) (https://stackoverflow.com/questions/39922986/how-do-i-pandas-group-by-to-get-sum)
- momento del dia (day_moment fue generada por el grupo 2 y puede reutilizarse el código que está aquí https://colab.research.google.com/drive/1KHgwiRqFLlavfhnCNdVvqG-xoPc0qMsM?usp=sharing#scrollTo=P04lwj2bSrFM )

**manteniendo** las features:

> - dia de la semana
> - fecha
> - presencia de errores en la conversación
> - hospital
> - cancelled
> - consulted
> - no correlation
> - y en general, todas las features que no sean body ni direction.
> - Debiendo quedar una fila por conversación.

* Cargar los datos, separando del dataset la etiqueta a predecir.
* Dividir el dataset en el conjunto de entrenamiento y conjunto de test
* Elegir y fundamentar si usar regresión lineal o polinomial(https://towardsdatascience.com/introduction-to-linear-regression-and-polynomial-regression-f8adc96f31cb, https://data36.com/polynomial-regression-python-scikit-learn/ ) . Tomarse un tiempito en hacer esta decisión les será muy útil ya que es una decisión interesante para comunicar en el video que tengan que hacer a partir de este práctico.
* Entrenar y evaluar un modelo de regresión lineal y uno polinomial, fijando la semilla aleatoria para hacer repetible el experimento.
* En cuanto a los hiper-parámetros:

        1.   Probar primero con los default y elegir alguna/s métrica/s para reportar los resultados. 
        2.   Luego usar grid-search (https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/) para explorar muchas combinaciones posibles de valores (Ejemplo de uso de grid search para encontrar el grado de polynomial regression https://stackoverflow.com/a/62460485/13597482 ). Reportar MAE y RMSE para las variaciones que busquen.

*   Para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando MAE y RMSE.

### Se evaluarán los siguientes aspectos:
  ***1-*** Que se apliquen los conceptos vistos con los profes en el teórico y en el práctico.

  ***2-*** Capacidad de análisis. Enfocarse en esto les permitirá completar el video que deben preparar a continuación.

  ***3-*** Criterio para elegir que solución aplicar en cada caso y con qué método implementarla.


  
## Deadline pautado para la entrega: Miércoles 21/09/2022

Es importante que vayan anotando **conclusiones y observaciones en lenguaje natural** (no muy extensas) a medida que vayan realizando las tareas para que les sirva como **punto de partida para el video** de 10 min que se realice a partir de este práctico. El video que sigue a este práctico serviría como informe de resultados y observaciones de este práctico, resaltando aprendizajes y problemáticas con las que se hayan encontrado.

Lo importante de este práctico **no es encontrar el mejor modelo posible** ni mucho menos, sino poder ejercitar todo el proceso, que implica tanto **preparar los datos hasta correr los experimentos**, y finalmente **analizar y explicar los resultados encontrados**. En mi experiencia, la primera reacción ante saber que hay que comunicar resultados es querer comunicar éxitos, pero los resultados exitosos son -la mayoría de las veces- poco informativos. En el proceso de realización del práctico se encontrarán con desafíos de las tareas propuestas y con problemas específicos de este tipo de datasets conversacionales. Esa información es la valiosa a comunicar ya que es una experiencia que ningún otro grupo pudo tener, y le puede servir a otro grupo en un futuro que se relacione con datos que tengan características parecidas.

# Resolución

In [19]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib
import matplotlib.pyplot as plt



In [20]:
file_key_marzo = 'marzo_anon_test.csv'
file_key_febrero = 'febrero_anon_test.csv'

file_key_procesado = 'dataset_post_entrega_2.csv'

df_marzo = pd.read_csv(file_key_marzo)
df_febrero = pd.read_csv(file_key_febrero)

In [21]:
df = pd.concat([df_febrero, df_marzo]).drop(['Unnamed: 0'], axis=1)
df['SentDate'] = pd.to_datetime(df.SentDate)

In [22]:
df.sample(2)

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id
3715,CityBell,whatsapp:+5492215137770,"Necesito tu DNI, por favor.",read,2022-03-02 19:29:00+00:00,2022-03-02,2,3,19,1,...,0,[],0,[],0,[],0,[],0,21
169577,CityBell,whatsapp:+5492215137770,*Escribe el número de la opción* que quieres e...,delivered,2022-03-18 13:06:13+00:00,2022-03-18,18,3,13,1,...,0,[],0,[],0,[],0,[],0,4689


In [23]:
df.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id'],
      dtype='object')

In [108]:
df.Dia.value_counts().reset_index().sort_values(by='index')

,index,Dia
22,1,4444
3,2,26475
6,3,23762
15,4,17148
23,5,3861
27,6,2598
2,7,29068
4,8,24688
13,9,20032
12,10,20222


### ¿Cómo se ven los mensajes de _Human Handoff_?
Buscamos una muestra de los mensajes que nos interesan, solo para ver que forma tienen

In [24]:
df_handoffed = df[df['Got_HH'] == 1]
df_handoffed[['conv_id', 'Body', 'Hospital', 'Direction']]

,conv_id,Body,Hospital,Direction
164,10,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
408,26,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
518,32,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
609,34,"Fuiste transferido a un humano, por favor escr...",Salud Consultorios,outbound-api
666,25,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
...,...,...,...,...
290219,4596,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290339,7341,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290660,7056,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290921,7325,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api


In [25]:
df_handoffed.Direction.value_counts()

outbound-api    2756
inbound            1
Name: Direction, dtype: int64

In [26]:
# Esto es un error ? 
df_handoffed[df_handoffed.Direction == 'inbound']

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id
197335,CityBell,whatsapp:+5492215137770,"Fuiste transferido a un humano, por favor escr...",received,2022-03-22 12:07:49+00:00,2022-03-22,22,3,12,1,...,0,[],0,[],0,[],0,[],0,4238


## ¿Cómo se vé una conversación que finaliza en _Human Handoff_?

Seleccionamos un valor de `conv_id` y `Hospital` como muestra para poder calcular el valor

In [113]:
# Nota:
# El valor de conv_id no es único.
df[df['conv_id'] == 32].Hospital.value_counts()

CityBell    56
Carra       22
Name: Hospital, dtype: int64

### Boceto: Calcular tiempo de respuesta para una única conversación

In [179]:
# Elegimos una conversación cualquiera, vemos que intercambio de mensajes hay

def get_sample(df, **kwargs):
    return df[(df['conv_id'] == kwargs['conv_id']) & (df['Hospital'] == kwargs['hospital'])][['Body','Direction','SentDate','Got_HH', 'Hospital']].sort_values(by='SentDate')


sample_conv1 = get_sample(df, hospital = 'CityBell', conv_id = 32)

# 7        Salud Consultorios
sample_conv2 = get_sample(df, hospital = 'Salud Consultorios', conv_id = 7)
sample_conv2

,Body,Direction,SentDate,Got_HH,Hospital
173,XXXX,inbound,2022-03-01 19:33:58+00:00,0,Salud Consultorios
171,"Hola, soy el asistente virtual de Salud Consul...",outbound-api,2022-03-01 19:33:59+00:00,0,Salud Consultorios
165,Sacar turno,inbound,2022-03-01 19:34:20+00:00,0,Salud Consultorios
163,"Necesito tu DNI, por favor.",outbound-api,2022-03-01 19:34:21+00:00,0,Salud Consultorios
159,XXXXXXXXX,inbound,2022-03-01 19:34:28+00:00,0,Salud Consultorios
...,...,...,...,...,...
146584,No se pudo enviar el mensaje,outbound-api,2022-03-16 14:26:36+00:00,0,Salud Consultorios
146591,"Necesito hacer estos análisis, me podrían pasa...",inbound,2022-03-16 14:27:12+00:00,0,Salud Consultorios
146534,201.0,inbound,2022-03-16 14:30:12+00:00,0,Salud Consultorios
146234,Buenas tardes! para laboratorio debe comunicar...,outbound-api,2022-03-16 14:47:47+00:00,0,Salud Consultorios


In [221]:
# tiempo_de_respuesta_HH (agrupando por conv_id, calcular el valor absoluto de la diferencia entre el SentDate 
# del mensaje con Got_HH igual a 1 y el del siguiente mensaje con Direction igual a outbound
conv = sample_conv.reset_index()

# Pre:conv tiene al menos un mensaje con Got_HH en 1

handoff = conv[conv.Got_HH == 1]
idx = handoff.index[0]
posteriores = conv[idx:]
to = posteriores[(posteriores.Direction == 'outbound-api') & (posteriores.Got_HH == 0)].head(1)
elapsed = to.SentDate - conv.iloc[i].SentDate

elapsed


def tiempo_de_respuesta_HH(data):
    """
    data es un conjunto de mensajes pertenecientes a una misma conversación (para un mismo día y hospital)
    Pre: data tiene al menos un mensaje con Got_HH en 1, los mensajes estan ordenados por SentDate
    """
    conv = data.reset_index()

    # Pre:conv tiene al menos un mensaje con Got_HH en 1
    handoff = conv[conv.Got_HH == 1]
    idx = handoff.index[0] 
    
    # Si es el último mensaje, no podemos obtener el tiempo buscado
    if idx == len(handoff):
        return None
    
    posteriores = conv[idx:]
    to = posteriores[(posteriores.Direction == 'outbound-api') & (posteriores.Got_HH == 0)]
    
    if not len(to):
        return None
    
    return to.head(1).SentDate - conv.iloc[idx].SentDate

print (tiempo_de_respuesta_HH(sample_conv))

# salud consultorios, conv id = 7 
print (tiempo_de_respuesta_HH(sample_conv2))


34   0 days 00:03:25
Name: SentDate, dtype: timedelta64[ns]
56   0 days 00:00:37
Name: SentDate, dtype: timedelta64[ns]


Ahora tenemos una muestra de como calcular el tiempo de respuesta para una conversación. En las siguientes secciones vamos a trabajar en filtrar los datos para considerar unicamente conversaciones con una ocurrencia de `Got_HH == 1`

### Boceto: Mensajes en la conversación

In [127]:
# TBD ¿Cómo podemos contar los mensajes en una conversación?

## Filtro de conversaciones

In [128]:
def with_human_handoff(df):
  """
  Filter conversations which where 'human handoffed'
  """
  c = df.groupby(by = ['conv_id', 'Fecha', 'Hospital'])[[ 'Got_HH']].sum()
  return c[c.Got_HH > 0]

In [129]:
hhs = with_human_handoff(df_handoffed)
len(hhs[hhs.Got_HH == 1]), len(hhs[hhs.Got_HH > 1])


(2472, 135)

In [130]:
# Nos quedamos únicamente con las conversaciones que tienen un único pasaje a humano
hhs = hhs[hhs.Got_HH == 1].reset_index().rename(columns = {"Got_HH": "Got_HH_sum"})
hhs

,conv_id,Fecha,Hospital,Got_HH_sum
0,7,2022-03-16,Salud Consultorios,1
1,10,2022-02-16,CityBell,1
2,11,2022-02-16,Salud Consultorios,1
3,13,2022-02-23,CityBell,1
4,17,2022-02-15,Salud Consultorios,1
...,...,...,...,...
2467,7410,2022-03-31,Salud Consultorios,1
2468,7422,2022-03-31,Salud Consultorios,1
2469,7424,2022-03-31,Salud Consultorios,1
2470,7438,2022-03-31,CityBell,1


### Seguimos mirando
A partir del índice del paso anterior podemos filtrar los grupos de interes en el dataframe general

In [131]:
x = df.merge(hhs, on = ['conv_id', 'Fecha', 'Hospital'])
x

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id,Got_HH_sum
0,CityBell,whatsapp:+5492215137770,Hola! Este es un mensaje automático. Me encuen...,received,2022-02-16 19:33:19+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
1,CityBell,whatsapp:+5492215137770,"Buenas tardes, por favor enviar los datos requ...",delivered,2022-02-16 19:33:17+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
2,CityBell,whatsapp:+5492215137770,Necesito hacerme un electrocardiograma,received,2022-02-16 19:22:00+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
3,CityBell,whatsapp:+5492215137770,"Fuiste transferido a un humano, por favor escr...",delivered,2022-02-16 19:21:47+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
4,CityBell,whatsapp:+5492215137770,Has seleccionado una práctica que requiere ate...,delivered,2022-02-16 19:21:47+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76403,CityBell,whatsapp:+5492215137770,"Necesito tu DNI, por favor.",read,2022-03-31 09:25:03+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76404,CityBell,whatsapp:+5492215137770,Sacar turno,received,2022-03-31 09:25:03+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76405,CityBell,whatsapp:+5492215137770,"Hola, soy el asistente virtual de Centro Medic...",read,2022-03-31 09:24:43+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76406,CityBell,whatsapp:+5492215137770,Tu sesión se ha reinciado por tiempo de inacti...,read,2022-03-31 09:24:43+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1


In [97]:
x.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id',
       'Got_HH_sum'],
      dtype='object')

In [101]:
x[x.Got_HH == 1][['Got_HH','Got_HH_sum']]
x.sample(3)

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id,Got_HH_sum
55637,CityBell,whatsapp:+5492215137770,Doy por finalizada la conversación!,read,2022-03-16 13:08:58+00:00,2022-03-16,16,3,13,1,...,[],0,[],0,[],0,[],0,4202,1
40505,CityBell,whatsapp:+5492215137770,Has seleccionado una práctica que requiere ate...,read,2022-03-07 09:46:08+00:00,2022-03-07,7,3,9,1,...,[],0,[],0,[],0,[],0,1110,1
67495,CityBell,whatsapp:+5492215137770,*Escribe el número de la opción* que quieres e...,read,2022-03-28 17:13:03+00:00,2022-03-28,28,3,17,1,...,[],0,[],0,[],0,[],0,6244,1


In [227]:
x.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id',
       'Got_HH_sum'],
      dtype='object')

In [242]:
def checkdf(df):
    x = df.reset_index()
    print(x[x.Got_HH_sum >= 1].SentDate)
    return df




g = x.sort_values(by='SentDate').groupby(by = ['conv_id', 'Hospital', 'Fecha'])

#checkdf(g.get_group((list(g.groups)[0])))
# tiempo_de_respuesta_HH(g.get_group((list(g.groups)[0])))


# A partir de cada grupo podemos armar un dataframe nuevo (¿podemos?)

result = pd.DataFrame()
for xxx in list(g.groups)[0:]:
    data = g.get_group(xxx)
    data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
    
    result = result.append(data)
    

result




/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

...

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tiempo_de_respuesta_HH'] = tiempo_de_respuesta_HH(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data)
/var/folders/s6/yjtm_slj60sft1f08wtzs2mw0000gn/T/ipykernel_47611/3447628287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id,Got_HH_sum,tiempo_de_respuesta_HH
55021,Salud Consultorios,whatsapp:+5493515266230,XXXX,received,2022-03-16 14:24:09+00:00,2022-03-16,16,3,14,1,...,0,[],0,[],0,[],0,7,1,NaT
55020,Salud Consultorios,whatsapp:+5493515266230,"Hola, soy el asistente virtual de Salud Consul...",read,2022-03-16 14:24:10+00:00,2022-03-16,16,3,14,1,...,0,[],0,[],0,[],0,7,1,NaT
55018,Salud Consultorios,whatsapp:+5493515266230,"Necesito tu DNI, por favor.",read,2022-03-16 14:24:29+00:00,2022-03-16,16,3,14,1,...,0,[],0,[],0,[],0,7,1,NaT
55019,Salud Consultorios,whatsapp:+5493515266230,Sacar turno,received,2022-03-16 14:24:29+00:00,2022-03-16,16,3,14,1,...,0,[],0,[],0,[],0,7,1,NaT
55017,Salud Consultorios,whatsapp:+5493515266230,XXXXXXXXX,received,2022-03-16 14:24:36+00:00,2022-03-16,16,3,14,1,...,0,[],0,[],0,[],0,7,1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76367,CityBell,whatsapp:+5492215137770,Has seleccionado una práctica que requiere ate...,read,2022-03-31 09:39:53+00:00,2022-03-31,31,3,9,1,...,0,[],0,[],0,[],0,7459,1,NaT
76366,CityBell,whatsapp:+5492215137770,Necesito tu email para transferirte a un opera...,read,2022-03-31 09:39:53+00:00,2022-03-31,31,3,9,1,...,0,[],0,[],0,[],0,7459,1,NaT
76365,CityBell,whatsapp:+5492215137770,XXXXXXXXXXXXXXXXXX,received,2022-03-31 09:41:27+00:00,2022-03-31,31,3,9,1,...,0,[],0,[],0,[],0,7459,1,NaT
76363,CityBell,whatsapp:+5492215137770,Has seleccionado una práctica que requiere ate...,read,2022-03-31 09:41:28+00:00,2022-03-31,31,3,9,1,...,0,[],0,[],0,[],0,7459,1,NaT


----

In [13]:
# df = df.filter(items = [2,5], axis=0)
#df_hh = df.filter(items = hhs.index.values, axis=0)

merged = pd.merge(hhs, df, left_index=True, right_on='conv_id', suffixes = ('_sum', '_ori')).sort_values(by='SentDate')
merged[merged.Got_HH_sum == 1].sample(1)[['Got_HH_sum', 'Got_HH_ori']]


,Got_HH_sum,Got_HH_ori
127502,1,0


In [14]:
merged.sample(3)

,conv_id,conv_id_sum,Fecha_sum,Hospital_sum,Got_HH_sum,Hospital_ori,Tel_hospital,Body,Status,SentDate,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id_ori
35777,1221,3009,2022-02-15,Los Cedros,1,CityBell,whatsapp:+5492215137770,*Escribe el número de la opción* que quieres e...,delivered,2022-02-05 16:50:48+00:00,...,0,[],0,[],0,[],0,[],0,1221
1266,68,125,2022-03-02,CityBell,1,CityBell,whatsapp:+5492215137770,*Escribe el número de la opción* que quieres e...,read,2022-02-16 17:30:06+00:00,...,0,[],0,[],0,[],0,[],0,68
138772,631,1417,2022-02-07,CityBell,1,Carra,whatsapp:+5493515266217,XXXXXXXXXXX,received,2022-03-15 10:38:19+00:00,...,0,[],0,[],0,[],0,[],0,631


In [15]:
# Todos estos mensajes pertenecen a conversaciones en las que se pidió HH
merged = merged.rename (columns = {"Got_HH_ori": "Got_HH"})
merged.head()

,conv_id,conv_id_sum,Fecha_sum,Hospital_sum,Got_HH_sum,Hospital_ori,Tel_hospital,Body,Status,SentDate,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id_ori
93996,1021,2411,2022-02-11,CityBell,1,CityBell,whatsapp:+5492215137770,Cancelar turno,received,2022-01-31 21:27:43+00:00,...,0,[],0,[],0,[],0,[],0,1021
93995,1021,2411,2022-02-11,CityBell,1,CityBell,whatsapp:+5492215137770,"Necesito tu DNI, por favor.",read,2022-01-31 21:27:44+00:00,...,0,[],0,[],0,[],0,[],0,1021
93994,1021,2411,2022-02-11,CityBell,1,CityBell,whatsapp:+5492215137770,XXXXXXXX,received,2022-01-31 21:27:52+00:00,...,0,[],0,[],0,[],0,[],0,1021
93993,1021,2411,2022-02-11,CityBell,1,CityBell,whatsapp:+5492215137770,*Escribe el número de la opción* que quieres e...,read,2022-01-31 21:27:55+00:00,...,0,[],0,[],0,[],0,[],0,1021
93992,1021,2411,2022-02-11,CityBell,1,CityBell,whatsapp:+5492215137770,1,received,2022-01-31 21:28:07+00:00,...,0,[],0,[],0,[],0,[],0,1021


In [16]:
merged.sort_values(by = 'SentDate').groupby(by= ['conv_id', 'Fecha', 'Hospital'])['Got_HH'].cumsum().value_counts()

KeyError: 'Fecha'

In [33]:
# index: ordinal
ds = merged.sort_values(by = 'SentDate').reset_index()
#ds.groupby(by ='conv_id')['Got_HH'].cumsum()

w = merged.groupby(by = ['conv_id', 'Got_HH', 'Direction']).first().reset_index()

w[w.conv_id == 10]
merged[merged.conv_id == 7].sort_values(by='SentDate')[['Body', 'Got_HH', 'Direction', 'SentDate', 'Status', 'Fail_HH']].tail(30)

,Body,Got_HH,Direction,SentDate,Status,Fail_HH
11030,*Escribe el número de la opción* que quieres e...,0,outbound-api,2022-03-02 12:51:41+00:00,read,0
81489,Tu sesión se ha reinciado por tiempo de inacti...,0,outbound-api,2022-03-09 16:47:55+00:00,read,0
81490,5.0,0,inbound,2022-03-09 16:47:55+00:00,received,0
81486,Necesito tu email para transferirte a un opera...,0,outbound-api,2022-03-09 16:47:56+00:00,read,0
81487,Si seleccionaste la especialidad de APTOS FÍSI...,0,outbound-api,2022-03-09 16:47:56+00:00,read,0
81488,"Disculpe, el asistente virtual de Salud Consul...",0,outbound-api,2022-03-09 16:47:56+00:00,read,0
146643,XXXX,0,inbound,2022-03-16 14:24:09+00:00,received,0
146642,"Hola, soy el asistente virtual de Salud Consul...",0,outbound-api,2022-03-16 14:24:10+00:00,read,0
146638,"Necesito tu DNI, por favor.",0,outbound-api,2022-03-16 14:24:29+00:00,read,0
146640,Sacar turno,0,inbound,2022-03-16 14:24:29+00:00,received,0


Quiero
* Agrupar por conv_id
* Para cada conversacion:
** Buscar el mensaje con Got_HH == 1
** Obtener el siguiente



Features que necesitamos

        dia de la semana
        fecha
        presencia de errores en la conversación
        hospital
        cancelled
        consulted
        no correlation
        y en general, todas las features que no sean body ni direction.
        Debiendo quedar una fila por conversación.



In [ ]:
def xyz(df):
  """
  Filter conversations which where 'human handoffed'
  The resulting dataframe is indexed by conv_id
  """
  return df.groupby(by = ['conv_id'])

gs = xyz(ds)
gs['Got_HH'].sum() > 0 

In [ ]:
import itertools
def handle_conversation(conv_id, hospital, conv_df):
    """
    Todos los mensajes en conv_df deben pertenecer a la misma conversación (y mismo hospital)
    """
    
    # human handoff request
    hh = conv_df[conv_df.Got_HH == 1]

    i = hh.index
    if len(i) == 1:
        msg = conv_df.iloc[i]
        
        haystack = conv_df[i:]
        to_df = haystack[conv_df.Direction == 'outbound-api']
        print(len(to_df))
        
    elif len(i) == 2:
        pass
    else: # TODO Filtrar antes
        print(f"La conversación {conv_id}/{hospital} no tiene HH")
    
    



In [ ]:
for (conv_id, hospital), indices in itertools.islice(gs.groups.items(), 1):

    conv_df = ds[(ds.conv_id == conv_id) & (ds.Hospital == hospital)].\
    sort_values(by='SentDate').\
    reset_index(drop = True)
    
    print(conv_df.columns)
    
    
    
    #handle_conversation(conv_id, hospital, conv_df)
